### Script to Generate Standalone HTML Chat History Report
Note: You must have run the `Chat-History-Prototyping` notebook first. Then, run `Chat-History-Report`, which calls this notebook to generate a standalone, shareable HTML report.

In [1]:
# Get stored visualization path list using iPython %store -r magic
%store -r

The script below relies heavily on the `nbconvert` and `nbformat` packages (used as libraries). The following references were heavily utilized:
- [The nbconvert library documentation](https://nbconvert.readthedocs.io/en/latest/nbconvert_library.html) for providing the implementation backbone.
- [The nbconvert cell removal guide](https://nbconvert.readthedocs.io/en/latest/removing_cells.html) for leveraging [notebook tags](https://stackoverflow.com/questions/48014844/what-is-the-usage-of-tag-over-the-cell-in-jupyter) to remove unwanted cells in the final report.
- [This post](https://stackoverflow.com/questions/26068316/embedding-image-in-ipython-notebook-for-distribution) for providing information on embedding images in HTML using base64.
- [This post](https://stackoverflow.com/questions/20184994/ipython-notebook-nbconvert-how-to-remove-red-outn-text-in-top-left-hand-co/30234937#30234937) for guidance on removing the left margin of a Jupyter Notebook using raw cells.
- [nbconvert documentation](https://nbconvert.readthedocs.io/_/downloads/en/5.3.0/pdf/) for reference.

In [2]:
import os
import base64
import nbformat
from nbconvert import HTMLExporter
from traitlets.config import Config

output_string = ""

for file_path in all_vis_paths:
    
    # Read image and convert to base64
    with open(file_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
        
    # Add to the HTML output string to embed image in base64 
    output_string += '<img src="data:image/png;base64,'+ encoded_string.decode("utf-8") + '" />\n'
    
# Remove the trailing newline char
output_string = output_string[:-1]

# 0-indexed cell position of the markdown cell
cell_number = 1 

# Read Markdown-formatted Jupyter Notebook to NotebookNode object
# (NOTE: all images must be in a single Markdown cell for this script to work!)
report_nbform = nbformat.read('Chat-History-Report.ipynb', as_version=4)

# Embed the image in base64 form
report_nbform.cells[cell_number]['source'] = output_string

# Instantiate the exporter
html_exporter = HTMLExporter()
#html_exporter.template_file = 'basic'

# Create a configuration object and configure tag removal 
c = Config()
c.TagRemovePreprocessor.remove_cell_tags = ("remove_cell",)

# Configure preprocessors
c.HTMLExporter.preprocessors = ["TagRemovePreprocessor"]
c.HTMLExporter.preprocessors = ['nbconvert.preprocessors.ExtractOutputPreprocessor']

# Create the new exporter using the custom config
html_exporter_with_figs = HTMLExporter(config=c)

# Process the notebook loaded earlier into HTML
(html_body, resources_with_fig) = html_exporter_with_figs.from_notebook_node(report_nbform)

# Write the report in HTML format
with open("Chat-History-Report.html", mode= "w") as f:
    f.write(html_body)
